In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3`
import $ivy.`sh.almond::almond-spark:0.6.0`

import org.apache.spark._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

import org.slf4j.LoggerFactory
import org.apache.log4j.{Level, Logger}

val spark = NotebookSparkSession
      .builder()
      .config("spark.sql.join.preferSortMergeJoin", false)
      .config("spark.sql.shuffle.partitions", 64)
      .master("local[*]")
      .getOrCreate()

import spark.implicits._

Logger.getRootLogger().setLevel(Level.ERROR)

def run[A](code: => A): A = {
    val start = System.currentTimeMillis()
    val res = code
    println(s"Took ${System.currentTimeMillis() - start}")
    res
}

Loading spark-stubs
Getting spark JARs


log4j:WARN No appenders could be found for logger (org.eclipse.jetty.util.log).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
22/01/03 18:42:55 INFO SparkContext: Running Spark version 2.4.3
22/01/03 18:42:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/03 18:42:55 INFO SparkContext: Submitted application: 1e5e0803-d0ba-4e29-b1a1-295ae5006e10
22/01/03 18:42:55 INFO SecurityManager: Changing view acls to: oscar
22/01/03 18:42:55 INFO SecurityManager: Changing modify acls to: oscar
22/01/03 18:42:55 INFO SecurityManager: Changing view acls groups to: 
22/01/03 18:42:55 INFO SecurityManager: Changing modify acls groups to: 
22/01/03 18:42:55 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(oscar); groups with view permissions: Set(); users  with modify permissions: Set(oscar); groups with modify permissions: Set()
22/01/03 18:42:55 INFO Utils: Successfully started service 'sparkDrive

22/01/03 18:42:56 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/mainargs_2.12/0.1.4/mainargs_2.12-0.1.4-sources.jar at spark://netrunner:41739/jars/mainargs_2.12-0.1.4-sources.jar with timestamp 1641231776420
22/01/03 18:42:56 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/mainargs_2.12/0.1.4/mainargs_2.12-0.1.4.jar at spark://netrunner:41739/jars/mainargs_2.12-0.1.4.jar with timestamp 1641231776420
22/01/03 18:42:56 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/os-lib_2.12/0.7.1/os-lib_2.12-0.7.1-sources.jar at spark://netrunner:41739/jars/os-lib_2.12-0.7.1-sources.jar with timestamp 1641231776421
22/01/03 18:42:56 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/os-lib_2.12/0.7.1/os-lib_2.12-0.7.1.jar at spark://netrunner:41739/jars/os-lib_2.12-0.7.1.j

22/01/03 18:42:56 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/github/alexarchambault/argonaut-shapeless_6.2_2.12/1.2.0-M11/argonaut-shapeless_6.2_2.12-1.2.0-M11.jar at spark://netrunner:41739/jars/argonaut-shapeless_6.2_2.12-1.2.0-M11.jar with timestamp 1641231776427
22/01/03 18:42:56 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/org/eclipse/jetty/jetty-server/9.4.19.v20190610/jetty-server-9.4.19.v20190610.jar at spark://netrunner:41739/jars/jetty-server-9.4.19.v20190610.jar with timestamp 1641231776427
22/01/03 18:42:56 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/io/argonaut/argonaut_2.12/6.2.3/argonaut_2.12-6.2.3.jar at spark://netrunner:41739/jars/argonaut_2.12-6.2.3.jar with timestamp 1641231776427
22/01/03 18:42:56 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/chuusai/shapeless_2.

Spark UI

import $ivy.$                                  

import $ivy.$                              


import org.apache.spark._

import org.apache.spark.sql.{functions => func, _}

import org.apache.spark.sql.functions._

import org.apache.spark.sql.types._


import org.slf4j.LoggerFactory

import org.apache.log4j.{Level, Logger}


spark: SparkSession = org.apache.spark.sql.SparkSession@3737774
import spark.implicits._


defined function run

In [2]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.0`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// if you want to have the plots available without an internet connection:
//init(offline=true)

// restrict the output height to avoid scrolling in output cells
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

import $ivy.$                                      

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// if you want to have the plots available without an internet connection:
//init(offline=true)

// restrict the output height to avoid scrolling in output cells


In [2]:
import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.expressions.Window

In [11]:
val drivers = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/drivers.csv")

load at cmd10.sc:4

1 / 1

drivers: DataFrame = [driverId: string, driverRef: string ... 7 more fields]

In [12]:
val races = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/races.csv")
    .select("raceId", "year")
    .where(col("year") === 2021)

load at cmd11.sc:4

1 / 1

races: Dataset[Row] = [raceId: string, year: string]

In [13]:
val driverWindow = Window.partitionBy("driverId")
val seasonWindow = Window.partitionBy("year")
val driverRaceWindow = Window.partitionBy("driverId", "raceId")
val raceDriverLapWindow = Window.partitionBy("driverId", "raceId").orderBy("lap")

driverWindow: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@439faf3f
seasonWindow: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@7ea985db
driverRaceWindow: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@3dade0f4
raceDriverLapWindow: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@2dc308d

In [14]:
val pitStops = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/pit_stops.csv")
    .withColumn("lap", (col("lap") - 1).cast(IntegerType)) 
    .join(races, "raceId")
    .select("raceId", "driverId", "lap")

load at cmd13.sc:4

1 / 1

pitStops: DataFrame = [raceId: string, driverId: string ... 1 more field]

In [7]:
pitStops.printSchema

root
 |-- raceId: string (nullable = true)
 |-- driverId: string (nullable = true)
 |-- lap: integer (nullable = true)



In [15]:
val overtakes = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/lap_times.csv")
    .withColumn("position", col("position").cast(IntegerType)) 
    .withColumn("lap", col("lap").cast(IntegerType)) 
    .join(races, "raceId")
    .join(pitStops, Seq("raceId", "driverId", "lap"), "left")
    .withColumn("positionNextLap", lead(col("position"), 1).over(raceDriverLapWindow))
    .withColumn("positionsGainedLap", when(col("positionNextLap") < col("position") , abs(col("position") - col("positionNextLap"))).otherwise(0))
    .withColumn("positionsLostLap", when(col("positionNextLap") > col("position"), abs(col("position") - col("positionNextLap"))).otherwise(0))
    .withColumn("positionsGained", sum(col("positionsGainedLap")).over(driverRaceWindow))
    .withColumn("positionsLost", sum(col("positionsLostLap")).over(driverRaceWindow))
    .withColumn("lapLeader", when(col("position") === 1, 1).otherwise(0))
    .withColumn("lapsLed", sum(col("lapLeader")).over(driverWindow))
    .withColumn("totalLaps", sum(col("lapLeader")).over(seasonWindow))
    .withColumn("percLapsLed", round(col("lapsLed") / col("totalLaps"), 2))
//     .sort(col("lap"))
    .where(col("lap") === 1)
    .select("raceId", "driverId", "positionsGained", "positionsLost", "lapsLed", "percLapsLed")


load at cmd14.sc:4

1 / 1

overtakes: DataFrame = [raceId: string, driverId: string ... 4 more fields]

In [17]:
val results = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/results.csv")

    .withColumn("position", col("position").cast(IntegerType))    
    .withColumn("grid", col("grid").cast(IntegerType))    
    .withColumn("points", col("points").cast(IntegerType))

    .join(races, "raceId")
    .join(overtakes, Seq("raceId", "driverId"), "left")
    .join(drivers, "driverId")

    .withColumn("podium", when(col("position") === 1 || col("position") === 2 ||col("position") === 3, lit(1)).otherwise(lit(0)))
    .withColumn("averagePoints", round(avg(col("points")).over(driverWindow), 2))
    .withColumn("maxAvgPoints", max(col("averagePoints")).over(seasonWindow))

    .select(
        col("code"),
        sum(col("points")).over(driverWindow).as("champPoints"),
        col("averagePoints"),
        round(col("averagePoints") / col("maxAvgPoints"),2).as("pointPercent"),
        sum(col("podium")).over(driverWindow).as("totalPodiums"),
        round(sum(col("podium")).over(driverWindow) / count(col("podium")).over(driverWindow), 2).as("podiumPercent"),
        round(avg(col("position") - col("grid")).over(driverWindow), 2).as("positionDelta"),
        round(avg(col("positionsLost")).over(driverWindow), 2).as("avgPositionsLost"),
        round(avg(col("positionsGained")).over(driverWindow), 2).as("avgPositionsWon"),
        sum(col("positionsLost")).over(driverWindow).as("totalPositionsLost"),
        sum(col("positionsGained")).over(driverWindow).as("totalPositionsWon"),
        col("lapsLed"),
        col("percLapsLed")
    )


    .dropDuplicates(Seq("code"))
    .sort(col("champPoints").desc)
    .show(30)

load at cmd16.sc:4

1 / 1

run at ThreadPoolExecutor.java:1149

1 / 1

run at ThreadPoolExecutor.java:1149

...

show at cmd16.sc:37

64 / 64

show at cmd16.sc:37

64 / 64

+----+-----------+-------------+------------+------------+-------------+-------------+----------------+---------------+------------------+-----------------+-------+-----------+
|code|champPoints|averagePoints|pointPercent|totalPodiums|podiumPercent|positionDelta|avgPositionsLost|avgPositionsWon|totalPositionsLost|totalPositionsWon|lapsLed|percLapsLed|
+----+-----------+-------------+------------+------------+-------------+-------------+----------------+---------------+------------------+-----------------+-------+-----------+
| VER|        395|        17.95|         1.0|          18|         0.82|        -1.21|            2.71|           3.38|                57|               71|    652|        0.5|
| HAM|        387|        17.59|        0.98|          17|         0.77|        -0.29|            4.32|           4.41|                95|               97|    297|       0.23|
| BOT|        226|        10.27|        0.57|          11|          0.5|        -0.78|            4.33|           5

In [6]:
val trace1 = Bar(
  Seq("giraffes", "orangutans", "monkeys"),
  Seq(20, 14, 23),
  name = "SF Zoo"
)

val trace2 = Bar(
  Seq("giraffes", "orangutans", "monkeys"),
  Seq(12, 18, 29),
  name = "LA Zoo"
)

val data = Seq(trace1, trace2)

val layout = Layout( 
  barmode = BarMode.Group
)

plot(data, layout)

xValue: Seq[String] = List("Product A", "Product B", "Product C")
yValue: Seq[Int] = List(20, 14, 23)
trace1: Bar = Bar(
  Strings(List("Product A", "Product B", "Product C")),
...
data: Seq[Bar] = List(
  Bar(
...
annotations: Seq[Annotation] = List(
  Annotation(
...
layout: Layout = Layout(
  Some("January 2013 Sales Report"),
...
res5_6: String = "plot-41be04cc-db4d-436d-a9c3-b5d102bd7222"